In [1]:
from deepface import DeepFace
import cv2
import os
import argparse
from time import perf_counter
from datetime import datetime

# parser = argparse.ArgumentParser(description="frame extractor")
# parser.add_argument("-video_path", "--video_path", help="specify path to video to extract frames from",
#                     default="C:\\Users\\milen\\test\\test2.mp4", type=str)
# args = parser.parse_args()



2024-05-30 21:04:22.965658: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 21:04:22.965709: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 21:04:22.965728: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-30 21:04:22.970583: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
face_videos_directory_path = "/tf/data/face_videos/source_videos/dapopov"
face_videos_list = [os.path.join(face_videos_directory_path,video_path) for video_path in os.listdir(person_videos_directory_path) if video_path.endswith("mp4")]

NameError: name 'person_videos_directory_path' is not defined

In [3]:
def extract_frames(video_path : str):
    video_name = video_path.split('\\')[-1].split(".")[0]
    # сохраняем данные о работе программы в файл
    f = open(f"{video_name}-extraction-log.txt", "w")
    f.write(f"{video_name}.mp4 frame extraction log {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}\n")
    
    isExist = os.path.exists(f"{video_name}_frames")
    if not isExist:
        os.mkdir(f"{video_name}_frames")
    else:
        print(f"Folder of frames from {video_name} already exists, skipping")
        return
    
    # открываем видеофайл
    cap = cv2.VideoCapture(video_path)
    if (cap.isOpened() == False):
        print("Error opening video stream or file")
    
    n = 200
    n_counter = 0
    saved_frame_counter = 1
    frame_counter = 0
    start_time = perf_counter()
    try: 
        while (cap.isOpened() and frame_counter < 2000):
            # print(f"n_counter: {n_counter}")
            if n_counter == 0:
                frame_counter += 1
                # захват кадра
                ret, frame = cap.read()
                if ret == True:
                    try:
                        # обнаружение лиц на текущем кадре
                        faces = DeepFace.extract_faces(
                            img_path=frame,
                            detector_backend="retinaface",
                            target_size=(224, 224)
                        )
                        # if (len(faces) > 1):
                        #     print("too many faces found on that frame, skipping")
                        #     continue
        
                        # сохраненяем текущий кадр
                        cv2.imwrite(f"{video_name}_frames/{saved_frame_counter}.jpg", frame)
                        saved_frame_counter += 1
        
                    except ValueError as e:
                        # если лицо в кадре не обнаружено
                        print(f"No face found for current frame, error message: {str(e)}")
                        n_counter = n / 2
                        continue
                # прерываем цикл
                else:
                    time_elapsed = perf_counter() - start_time
        
                    print(f"Time elapsed: {time_elapsed}, FPS: {frame_counter / time_elapsed}\n")
                    f.write(f"Time elapsed: {time_elapsed}, FPS: {frame_counter / time_elapsed}\n")
                    print(f"frames processed: {frame_counter}")
                    f.write(f"frames processed: {frame_counter}")
                    print("video ended, exiting")
                    f.write("video ended, exiting")
                    break
                n_counter = n
            else:
                n_counter -= 1
                _ = cap.grab()
    except KeyboardInterrupt as e:
        print("User interrupted the process")
        
    # когда все сделано, освобождаем видеозахват и закрываем файл для записи данных
    cap.release()
    f.close()

In [5]:
videos_basedir = "/tf/data/face_videos/source_videos"
person_list = [ name for name in os.listdir(videos_basedir) if os.path.isdir(os.path.join(videos_basedir, name)) ]
for person in person_list:
    # print(f"current person: {person}")
    face_videos_directory_path = os.path.join(videos_basedir, person)
    # print(face_videos_directory_path)
    face_videos_list = [os.path.join(face_videos_directory_path,video_path) for video_path in os.listdir(face_videos_directory_path) if video_path.endswith(("mp4", "mkv"))]
    # print(face_videos_list)
    for face_video in face_videos_list:
        print(f"Processing a video {face_video}")
        extract_frames(face_video)

Processing a video /tf/data/face_videos/source_videos/adpiskunov/adpiskunov_1.mp4
Folder of frames from /tf/data/face_videos/source_videos/adpiskunov/adpiskunov_1 already exists, skipping
Processing a video /tf/data/face_videos/source_videos/adpiskunov/adpiskunov_2.mp4
Folder of frames from /tf/data/face_videos/source_videos/adpiskunov/adpiskunov_2 already exists, skipping
Processing a video /tf/data/face_videos/source_videos/dapopov/dapopov_1.mp4
Folder of frames from /tf/data/face_videos/source_videos/dapopov/dapopov_1 already exists, skipping
Processing a video /tf/data/face_videos/source_videos/dapopov/dapopov_2.mp4
Folder of frames from /tf/data/face_videos/source_videos/dapopov/dapopov_2 already exists, skipping
Processing a video /tf/data/face_videos/source_videos/fivanov/fiivanov_1.mp4
No face found for current frame, error message: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
No f

[ WARN:0@1013.172] global cap_ffmpeg_impl.hpp:1541 grabFrame packet read max attempts exceeded, if your video have multiple streams (video, audio) try to increase attempt limit by setting environment variable OPENCV_FFMPEG_READ_ATTEMPTS (current value is 4096)
[ WARN:0@1013.183] global cap_ffmpeg_impl.hpp:1541 grabFrame packet read max attempts exceeded, if your video have multiple streams (video, audio) try to increase attempt limit by setting environment variable OPENCV_FFMPEG_READ_ATTEMPTS (current value is 4096)
[ WARN:0@1013.195] global cap_ffmpeg_impl.hpp:1541 grabFrame packet read max attempts exceeded, if your video have multiple streams (video, audio) try to increase attempt limit by setting environment variable OPENCV_FFMPEG_READ_ATTEMPTS (current value is 4096)
[ WARN:0@1013.207] global cap_ffmpeg_impl.hpp:1541 grabFrame packet read max attempts exceeded, if your video have multiple streams (video, audio) try to increase attempt limit by setting environment variable OPENCV_F

Time elapsed: 127.2723209860269, FPS: 4.510014397105392

frames processed: 574
video ended, exiting
Processing a video /tf/data/face_videos/source_videos/iiromanova/iiromanova_2.mp4
No face found for current frame, error message: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
No face found for current frame, error message: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


[h264 @ 0x38e1d240] out of range intra chroma pred mode
[h264 @ 0x38e1d240] error while decoding MB 105 42
[h264 @ 0x39f71d00] cbp too large (49) at 51 26
[h264 @ 0x39f71d00] error while decoding MB 51 26


No face found for current frame, error message: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
No face found for current frame, error message: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
No face found for current frame, error message: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
No face found for current frame, error message: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


...

No face found for current frame, error message: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
No face found for current frame, error message: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


[h264 @ 0x39cdd200] out of range intra chroma pred mode
[h264 @ 0x39cdd200] error while decoding MB 59 26
[h264 @ 0x382894c0] out of range intra chroma pred mode
[h264 @ 0x382894c0] error while decoding MB 53 60
[h264 @ 0x39f74c00] mb_type 134 in I slice too large at 57 60
[h264 @ 0x39f74c00] error while decoding MB 57 60
[h264 @ 0x382aa180] out of range intra chroma pred mode
[h264 @ 0x382aa180] error while decoding MB 88 46
[h264 @ 0x38f10c40] out of range intra chroma pred mode
[h264 @ 0x38f10c40] error while decoding MB 106 66
[h264 @ 0x382aa180] out of range intra chroma pred mode
[h264 @ 0x382aa180] error while decoding MB 97 52
[h264 @ 0x396b6400] out of range intra chroma pred mode
[h264 @ 0x396b6400] error while decoding MB 5 45
[h264 @ 0x38f10c40] negative number of zero coeffs at 4 67
[h264 @ 0x38f10c40] error while decoding MB 4 67
[h264 @ 0x396b6400] mb_type 174 in P slice too large at 99 27
[h264 @ 0x396b6400] error while decoding MB 99 27
[h264 @ 0x39f78180] out of range

Time elapsed: 183.4839973009657, FPS: 4.550805586769311

frames processed: 835
video ended, exiting
Processing a video /tf/data/face_videos/source_videos/saslastnikov/saslastnikov_4.mp4
No face found for current frame, error message: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
No face found for current frame, error message: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
No face found for current frame, error message: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
No face found for current frame, error message: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
No face found for current frame, error message: Fa

[h264 @ 0x3884ec80] corrupted macroblock 33 62 (total_coeff=-1)
[h264 @ 0x3884ec80] error while decoding MB 33 62


No face found for current frame, error message: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
No face found for current frame, error message: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
No face found for current frame, error message: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
No face found for current frame, error message: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
No face found for current frame, error message: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
No face found for current frame, error messag